# Viết chương trình sử dụng HMM để nhận dạng khẩu lệnh đơn lẻ

In [23]:
import librosa
import numpy as np
import os
import IPython.display as dsp

from hmmlearn import hmm

In [2]:
def normalize(feature):
    normalized = np.full_like(feature, 0)
    for i in range(feature.shape[0]):        
        normalized[i] = feature[i] - np.mean(feature[i]) # đưa trung bình về 0
        normalized[i] = normalized[i] / np.max(np.abs(normalized[i])) # đưa khoảng giá trị về [-1, 1]
    return normalized

In [3]:
def get_template(path):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y = y)
    delta = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order = 2)
    # print(np.transpose(np.concatenate([mfcc, delta, delta2], axis = 0),[1,0]).shape)
    return normalize(np.transpose(np.concatenate([mfcc, delta, delta2], axis = 0),[1,0]))

In [4]:
train_path = os.path.join(os.getcwd(), 'wav', 'train')
train_path

'/home/khiemvn/bt/wav/train'

Quá trình huấn luyện chỉ gần gọi `GaussianHMM` từ thư viện `hmmlearn`. Các tham số được chọn bao gồm:
- `n_components=5` số âm trong một khẩu lệnh. Chọn 5 để có thể xử lý khẩu lệnh phức tạp về âm như **phải** hoặc **xuống**
- `covariance_type='diag'` ma trận hiệp phương sai là ma trận chéo
- `n_iter=1800` số lần lặp, chọn tùy ý

In [5]:
def train(train_dir):
    train_files = [x for x in os.listdir(train_dir) if x.endswith('.wav')]
    X = np.array([])
    for file_name in train_files:
        file_path = os.path.join(train_dir, file_name)
        try:
            t = get_template(file_path)
            if len(X) == 0:
                X = t
            else:
                X = np.append(X, t, axis=0)
        except:
            pass
    model = hmm.GaussianHMM(n_components=5, covariance_type='diag', n_iter=100)
    # fit hmm model
    np.seterr(all='ignore')
    model.fit(X)
    return model

Mỗi mô hình HMM chỉ nhận dạng được 1 khẩu lệnh, do đó ta cần huấn luyện 8 mô hình HMM tất cả khẩu lệnh

In [6]:
hmm_models = []
for digit in os.listdir(train_path):
    label = digit
    path = os.path.join(train_path, digit)
    hmm_models.append((train(path), label))
    print(path)

/home/khiemvn/bt/wav/train/a
/home/khiemvn/bt/wav/train/trai
/home/khiemvn/bt/wav/train/len
/home/khiemvn/bt/wav/train/ban
/home/khiemvn/bt/wav/train/nhay
/home/khiemvn/bt/wav/train/xuong
/home/khiemvn/bt/wav/train/b
/home/khiemvn/bt/wav/train/phai


In [7]:
def predict(test_file):
    t = get_template(test_file)
    max_score = -float('inf')
    predicted_label = ""
    for item in hmm_models:
        model, label = item
        score = model.score(t)
        if score > max_score:
            max_score = score
            predicted_label = label
    return predicted_label

In [11]:
test_path = os.path.join(os.getcwd(), 'wav', 'test')

In [12]:
def test():
    test_paths = []
    for path in os.listdir(test_path):
        label = path
        test_arr = os.listdir(os.path.join(test_path, path))
        # print(arr)
        # test_arr = np.random.choice(arr,10)
        # print(test_arr)
        for test_file in test_arr:
            if not test_file.startswith('.'):
                test_paths.append({
                "test_path" : os.path.join(train_path,path,test_file),
                "label": label})

    correct = 0
    for test in test_paths:
        predict_label = predict(test["test_path"])
        if predict_label == test["label"]:
            correct +=1
    print('độ chính xác: {:.2f}%\n'.format(correct/len(test_paths)*100))

In [13]:
test()

độ chính xác: 83.33%



In [51]:
commands = ['a', 'b', 'len', 'xuong', 'trai', 'phai', 'ban', 'nhay']
num = [1, 2, 3]
cmd = np.random.choice(commands, 1)[0]
no = np.random.choice(num, 1)[0]
test = os.path.join(test_path, cmd, str(no) + '.wav')
test

'/home/khiemvn/bt/wav/test/b/2.wav'

In [52]:
dsp.Audio(test)

In [53]:
predict(test)

'b'

Xuất mô hình đã huấn luyện

In [10]:
import pickle

with open('model', 'wb') as handle:
    pickle.dump(hmm_models, handle, protocol=pickle.HIGHEST_PROTOCOL)